## Deliverable 2. Create a Customer Travel Destinations Map.

In [45]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import json
import sys
sys.path.append('../')
import config

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,10.42,79,100,6.91,overcast clouds
1,1,Kapaa,US,22.0752,-159.3190,64.72,59,0,9.22,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,50.11,71,75,25.32,light intensity shower rain
3,3,Elko,US,41.0002,-115.5012,37.06,79,100,3.00,overcast clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,87.76,58,0,10.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
chosen_temp_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & \
                                 (city_data_df["Max Temp"]<= max_temp)]

chosen_temp_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,87.76,58,0,10.36,clear sky
7,7,Sur,OM,22.5667,59.5289,75.72,64,0,12.26,clear sky
12,12,Puerto Ayora,EC,-0.7393,-90.3518,80.56,82,92,5.99,overcast clouds
14,14,Beloha,MG,-25.1667,45.0500,76.39,87,21,16.31,few clouds
16,16,Harindanga,IN,22.0383,88.3214,75.18,49,23,4.85,few clouds
19,19,Port Hedland,AU,-20.3167,118.5667,82.45,83,1,6.91,clear sky
20,20,Cabo San Lucas,MX,22.8909,-109.9124,80.51,42,0,3.44,clear sky
25,25,Rikitea,PF,-23.1203,-134.9692,79.77,69,82,18.79,broken clouds
27,27,Mahebourg,MU,-20.4081,57.7000,80.89,89,75,19.57,broken clouds
28,28,Saint-Pierre,RE,-21.3393,55.4781,80.28,83,64,2.30,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
chosen_temp_df.count()

City_ID                223
City                   223
Country                220
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_cities_df = chosen_temp_df.dropna()

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,87.76,58,0,10.36,clear sky
7,7,Sur,OM,22.5667,59.5289,75.72,64,0,12.26,clear sky
12,12,Puerto Ayora,EC,-0.7393,-90.3518,80.56,82,92,5.99,overcast clouds
14,14,Beloha,MG,-25.1667,45.0500,76.39,87,21,16.31,few clouds
16,16,Harindanga,IN,22.0383,88.3214,75.18,49,23,4.85,few clouds
19,19,Port Hedland,AU,-20.3167,118.5667,82.45,83,1,6.91,clear sky
20,20,Cabo San Lucas,MX,22.8909,-109.9124,80.51,42,0,3.44,clear sky
25,25,Rikitea,PF,-23.1203,-134.9692,79.77,69,82,18.79,broken clouds
27,27,Mahebourg,MU,-20.4081,57.7000,80.89,89,75,19.57,broken clouds
28,28,Saint-Pierre,RE,-21.3393,55.4781,80.28,83,64,2.30,broken clouds


In [7]:
preferred_cities_df.count()

City_ID                220
City                   220
Country                220
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Arraial Do Cabo,BR,87.76,clear sky,-22.9661,-42.0278,
7,Sur,OM,75.72,clear sky,22.5667,59.5289,
12,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,
14,Beloha,MG,76.39,few clouds,-25.1667,45.0500,
16,Harindanga,IN,75.18,few clouds,22.0383,88.3214,
19,Port Hedland,AU,82.45,clear sky,-20.3167,118.5667,
20,Cabo San Lucas,MX,80.51,clear sky,22.8909,-109.9124,
25,Rikitea,PF,79.77,broken clouds,-23.1203,-134.9692,
27,Mahebourg,MU,80.89,broken clouds,-20.4081,57.7000,
28,Saint-Pierre,RE,80.28,broken clouds,-21.3393,55.4781,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [56]:
# test = json.dumps(hotels, indent=5)
# print(test)

In [42]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Arraial Do Cabo,BR,87.76,clear sky,-22.9661,-42.0278,Pousada Porto Praia
7,Sur,OM,75.72,clear sky,22.5667,59.5289,Sur Plaza Hotel
12,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Beloha,MG,76.39,few clouds,-25.1667,45.0500,
16,Harindanga,IN,75.18,few clouds,22.0383,88.3214,Hazi Home
19,Port Hedland,AU,82.45,clear sky,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...
20,Cabo San Lucas,MX,80.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
25,Rikitea,PF,79.77,broken clouds,-23.1203,-134.9692,People ThankYou
27,Mahebourg,MU,80.89,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
28,Saint-Pierre,RE,80.28,broken clouds,-21.3393,55.4781,Lindsey Hôtel


In [20]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Arraial Do Cabo,BR,87.76,clear sky,-22.9661,-42.0278,Pousada Porto Praia
7,Sur,OM,75.72,clear sky,22.5667,59.5289,Sur Plaza Hotel
12,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Harindanga,IN,75.18,few clouds,22.0383,88.3214,Hazi Home
19,Port Hedland,AU,82.45,clear sky,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "../vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [40]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))